In [1]:
import os, json, re
import random, time
import pandas as pd, numpy as np
from tqdm import tqdm
from datetime import datetime, timedelta

# Util built for sharktank project
import sharktank_utils as su
from sharktank_utils import load_facts, OrchestratorResponse, PitchOrchestrator, metrics_calculation

# Agno for llm agents
from agno.agent import Agent
from agno.models.groq import Groq
from agno.storage.agent.sqlite import SqliteAgentStorage

# Other LLM utils
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List, Dict, Optional

In [2]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
reference_models = su.REFERENCE_MODELS
agent_storage = su.agent_storage

In [3]:
class PitchEditor(PitchOrchestrator):
    def __init__(self, orchestrator="llama-3.3-70b-versatile", reference=reference_models, editor='llama3-70b-8192', iterations=2):
        super().__init__(orchestrator=orchestrator, reference=reference)
        self.editor = editor
        self.iterations = iterations
    
    def create_editor_agent(self):
        instruction = """Given a pitch for a sharktank viewing. Give 2-3 points of feedback. Do not introduce new facts. 
        Answer shortly in the following format:
        - ...
        - ...
        """
        editor = Agent(
            name="Pitch Editor", 
            model=Groq(id=self.editor, max_tokens=1024),
            instructions=instruction,
            storage=SqliteAgentStorage(table_name="director_agent", db_file=agent_storage),
            add_datetime_to_instructions=True,
        )
        return editor

    def edit_pitch(self, pitch):
        """Give feedback on how to improve the pitch"""
        self.editor_agent = self.create_editor_agent()
        editor_prompt = f"""here is the pitch: {pitch}. Give short pointers."""
        editor_response = self.editor_agent.run(editor_prompt)
        self.logs.append(editor_response.metrics)
        return_prompt = f"""Edit a pitch with the given facts. With this pitch: {pitch}. {editor_response.content}."""
        return return_prompt
    
    def orchestrate_with_edit(self, goal, facts, verbose=False):
        """Full pipeline: generate subtasks, create agents, execute, and synthesize pitch."""
        try:
            for i in tqdm(
                range(self.iterations), 
                bar_format='[{elapsed}<{remaining}] {n_fmt}/{total_fmt} | {l_bar}{bar} {rate_fmt}{postfix}', 
                desc="Running Pitch Editing Iterations",
                colour='yellow'
            ):
                subtasks = self.generate_subtasks(goal, facts)
                if verbose:
                    print("\t>>> Subtasks:", subtasks)
                time.sleep(1)
                self.create_agents(subtasks, facts)
                time.sleep(1)
                agent_outputs = self.run_agents(subtasks)
                if verbose:
                    print("\t>>> Agent Outputs:", agent_outputs)
                time.sleep(1)
                pitch_initial_offer = self.synthesize_pitch(agent_outputs)
                if verbose:
                    print("\t>>> Pitch Draft:", pitch_initial_offer)
                time.sleep(1)
                if i != self.iterations - 1:
                    goal = self.edit_pitch(pitch_initial_offer)
                    if verbose:
                        print("\t>>> New goal:", goal)
                    time.sleep(1)
        except Exception as e:
            print(e, "for case:", facts['product_description'])
            return ""
        return pitch_initial_offer


In [4]:
facts_store = load_facts()
pitches = {k:"" for k,_ in facts_store.items()}
metrics = {k:"" for k,_ in facts_store.items()}
time_stamps = {k:"" for k,_ in facts_store.items()}

In [12]:
orcheditor = PitchEditor(iterations=2)
goal = 'create a pitch with the given facts'
# for case, facts in tqdm(facts_store.items()):
case = 'facts_shark_tank_transcript_37_TIK PIK.txt'
facts = facts_store[case]
lean_facts = {}
lean_facts['facts'], lean_facts['product_description'] = facts['facts'], facts['product_description']
try:
    pitch = orcheditor.orchestrate_with_edit(goal, lean_facts, verbose=False)
    pitches[case] = pitch
    metrics[case] = orcheditor.logs
    time_stamps[case] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
except Exception as e:
    print(case, "error:", e)
    pitch = ""

# reset agents
orcheditor.logs = []
orcheditor.agents = {}

[00:38<00:00] 2/2 | Running Pitch Editing Iterations: 100%|██████████ 19.12s/it


In [13]:
pitch

'{\n    "Pitch": "TIK PIK is introducing a revolutionary nano-suction material guitar pick that offers a secure grip without adhesive or residue and resists sweat and dirt, addressing major pain points for guitar players. With a unique value proposition, we\'re seeking an investment to expand our sales channels and drive growth. Our current sales are 85% direct to customers and 15% wholesale, with expected sales of $120,000 this year and a potential profit of up to $20,000. We\'re looking to allocate the investment into marketing, production, and product development to further enhance our product offerings and reach a wider audience.",\n    "Initial_Offer": {\n        "Valuation": "$10 million",\n        "Equity_Offered": "10%",\n        "Funding_Amount": "$1 million",\n        "Key_Terms": "Investment to be allocated into marketing, production, and product development"\n    }\n}'

In [15]:
calculated_metrics = []
for product, metric in metrics.items():
    calculated_metrics.append(metrics_calculation(metric))
input_lengths, output_lengths, latencies = zip(*calculated_metrics)

In [16]:
for k, v in pitches.items():
    if v == "":
        print(k)

In [17]:
timestamp = datetime.now().strftime('%Y-%d-%m')
framework = "orchestrator"
layer = "orchestrator_1"
pitches_df = pd.DataFrame(
    data={
        'scenario_name':list(pitches.keys()),
        'framework':len(pitches) * [framework],
        'layer':len(pitches) * [layer],
        'model_name': len(pitches) * ["groq/" + orcheditor.orchestrator + "/groq" + orcheditor.editor],
        'model_identity':len(pitches) * ["groq/" + orcheditor.orchestrator + "/groq" + orcheditor.editor],
        'latency':latencies,
        'input_length':input_lengths,
        'output_length':output_lengths,
        'time_stamp':list(time_stamps.values()),
        'response':list(pitches.values()),
    }
)


In [19]:
pitches_df.to_excel(f"{timestamp}-orchestrator-editor-pitches.xlsx")